In [1]:
import numpy as np

### operaciones básicas de cambios de filas en matrices

In [22]:
def swap_rows(A, i , j):
    '''intercambia la fija j por la i 
    en la matriz de coeficientes A
    de un sistema lienal
    return una nueva matriz A con las filas intercambiadad
    pero con la matriz A original sin modificar'''

    A_copy = np.copy(A)

    ai = np.copy(A[i])
    

    A_copy[i] = A_copy[j]

    A_copy[j] = ai


    return A_copy


def k_row(A: np.array, i: int, k: float):

    '''miltiplicar la fila i de la matriz A
    por un valor k que puede ser un entero, fraccion...
    '''

    A_copy = np.copy(A)
    
    A_copy[i] = A_copy[i] * float(k)

    return A_copy

def row_lineal_comb(A, i, j, k):

    ''' i es la posicion de la linea a la que multiplicas por k
    j es el indice de la linea que estamos modificando
    k es el valor por el cual multiplicamos a A[i]

    devuelve uan copia de la matriz 
    '''

    A_copy = np.copy(A)
    ai = np.copy(A[i])

    A_copy[j] = A_copy[j] + ai * k

    return A_copy


In [20]:
A = np.array([[1, 2, 3],
             [3, 4, 4],
             [4, 5, 3]], dtype = float)

A_swap = swap_rows(A, 0, 1)
A_k = k_row(A, 0, 1/2)
A_cl = row_lineal_comb(A, 0, 1, -2)
print(A)
print()
print(A_k)
print()
print(A_swap)
print()
print(A_cl)

[[1. 2. 3.]
 [3. 4. 4.]
 [4. 5. 3.]]

[[0.5 1.  1.5]
 [3.  4.  4. ]
 [4.  5.  3. ]]

[[3. 4. 4.]
 [1. 2. 3.]
 [4. 5. 3.]]

[[ 1.  2.  3.]
 [ 1.  0. -2.]
 [ 4.  5.  3.]]


### Descomposición Gauss

In [23]:
def gauss_U(A: np.array, func = row_lineal_comb) -> np.array:

    '''se le pasa la matriz A 
    una funcion func que se encargue de hacer al combinacion lineal: A[j] = A[j] + A[i] * k

    puede devolverte la matriz triangular superior de A 
    tambien puede devolver la triangular de A ampliada con el vector/matriz solución

    A -> U
    A|B -> U|C
    '''
    A_copy = np.copy(A)
    
    for i in range(len(A_copy)): #iteramos cada fila
        #print(i)
        for j in range(i +1, len(A_copy)): #iteramos solo sobre las filas que estan debajo del pivote
            #print(j)
            if A_copy[i, i] == 0:
                continue
            k =  A_copy[j, i]/ A_copy[i, i]
            #print(k)
            A_copy = func(A_copy, i, j, -k)
            #print(A)


    return A_copy


A = np.array([[4, -2, 1],
             [-2, 4, -2],
             [1, -2, 4]], dtype = float)

b = np.array([11, -16, 17])

A_ampliada = np.column_stack((A, b))


print(A_ampliada)
print(gauss_U(A_ampliada))
print(A)
print(gauss_U(A))

[[  4.  -2.   1.  11.]
 [ -2.   4.  -2. -16.]
 [  1.  -2.   4.  17.]]
[[  4.   -2.    1.   11. ]
 [  0.    3.   -1.5 -10.5]
 [  0.    0.    3.    9. ]]
[[ 4. -2.  1.]
 [-2.  4. -2.]
 [ 1. -2.  4.]]
[[ 4.  -2.   1. ]
 [ 0.   3.  -1.5]
 [ 0.   0.   3. ]]


### Sustitución hacia atrás

#### para solo un sistema de eccuaciones

In [24]:
def backward_sustitusion(U, c):
    '''
    the input is an upper triangular matrix U which dimension is nxn
    and c vector which contains the results, variable dimension
    the output is vector x, which initialy contained the unkwons, with the solution
    '''
    A_copy = np.copy(U)
    b_copy = np.copy(c)
    
    n = len(c)

    x = np.zeros((n,))


    for i in range(n-1, -1, -1):
        total_sum = 0
        for j in range(i+1, n):
            total_sum += A_copy[i, j] * x[j]

        x[i] = (b_copy[i] - total_sum) / A_copy[i, i]


    return x

A = np.array([[4, -2, 1],
             [-2, 4, -2],
             [1, -2, 4]], dtype = float)

b = np.array([11, -16, 17])

A_ampliada = np.column_stack((A, b))
U_extended = gauss_U(A_ampliada)

A_echelon = U_extended[:, 0:-1]
b_transformed = U_extended[:, -1]
print(b_transformed)

backward_sustitusion(A_echelon, b_transformed)


[ 11.  -10.5   9. ]


array([ 1., -2.,  3.])

#### para p sistemas de ecuaciones a la vez

In [25]:
def backward_sustitusion_p_systems(A, B):
    '''
    the input is an upper triangular matrix A which dimension is nxn
    and B matrix which contains the results, its dimension is nxp
    the output is matrix X, which initialy contained the unkwons, with the solution
    '''
    A_copy = np.copy(A)
    B_copy = np.copy(B)
    
    n = len(A)
    p = len(B[0])

    X = np.zeros(B.shape)

    for s in range(p):
        for i in range(n-1, -1, -1):
            total_sum = 0
            for j in range(i+1, n):
                total_sum += A_copy[i, j] * X[j, s]

            X[i, s] = (B_copy[i, s] - total_sum) / A_copy[i, i]

    return X


A = np.array([[6, -4, 1],
              [-4, 6, -4],
              [1, -4, 6]])

B = np.array([[-14, 22], 
              [36, -18],
              [6, 7]])

A_concat_B = np.column_stack((A, B))
#print(A_concat_B)
gauss_A_B = gauss_U(A_concat_B)
#print(gauss_A_B)

A_triang = gauss_A_B[:, 0:-2]
B_mod = gauss_A_B[:, 3:]

backward_sustitusion_p_systems(A_triang, B_mod)

array([[11.94444444,  3.        ],
       [25.66666667, -1.        ],
       [17.        ,  0.        ]])

### Descomposición LU Doolite

In [ ]:
def LU_decomposition(A: np.array, func = row_lineal_comb) -> np.array:

    '''se le pasa la matriz A
    una funcion func que se encargue de hacer al combinacion lineal: A[j] = A[j] - A[i] * k

    objetivo -> A = LU

    la salida es una matriz rectangular U triangular superior y una matriz cuadrada L triangular inferior
    U es la escalonada aplicando gauss
    L contiene los valores de k para calcular la escalonada y una diagonal de 1 para que L * U  = A
    '''
    A_copy = np.copy(A)
    n = len(A_copy)

    L = np.identity((n))


    for i in range(n): #iteramos cada fila
        for j in range(i +1, n): #iteramos solo sobre las filas que estan debajo del pivote
            if A_copy[i, i] == 0:
                continue

            k =  A_copy[j, i]/ A_copy[i, i]

            L[j, i] = k

            A_copy = func(A_copy, i, j, -k)

    U = A_copy
    return L, U 
    

A = np.array([[4, -2, 1],
             [-2, 4, -2],
             [1, -2, 4]], dtype = float)

b = np.array([11, -16, 17])

L1, U1 = LU_decomposition(A)
print(L1)
print(U1)
print(L1@U1)

[[ 1.    0.    0.  ]
 [-0.5   1.    0.  ]
 [ 0.25 -0.5   1.  ]]
[[ 4.  -2.   1. ]
 [ 0.   3.  -1.5]
 [ 0.   0.   3. ]]
[[ 4. -2.  1.]
 [-2.  4. -2.]
 [ 1. -2.  4.]]


### Sustitución hacia delante y hacia atrás

#### solo un sistema

In [27]:
def forward_substitution(L, b):
    """Resuelve Lc = b para c usando sustitución hacia adelante."""
    n = len(b)
    c = np.zeros_like(b, dtype=float)
    for i in range(n):
        c[i] = (b[i] - np.dot(L[i, :i], c[:i])) / L[i, i]
    return c

#### más de un sistema

In [28]:
def forward_substitution_p_systems(L, B):
    """
    Resuelve L * C = B usando sustitución hacia adelante, 
    donde L es triangular inferior y B es una matriz (varios sistemas).
    """
    n = L.shape[0]  # Tamaño de la matriz
    m = B.shape[1]  # Número de sistemas (columnas de B)
    C = np.zeros_like(B, dtype=float)  # Matriz solución C con mismas dimensiones que B

    # Iterar sobre cada fila
    for i in range(n):
        for j in range(m):  # Iterar sobre cada columna de B
            C[i, j] = (B[i, j] - np.dot(L[i, :i], C[:i, j])) / L[i, i]

    return C


#### combina sustituion hacia delante y hacia atras

In [29]:
def forward_backward_sustitution(L, U, b, func= backward_sustitusion):
    '''
    the input is a lower trriangular matrix L,
    an upper triangular matrix U,
    yhe solution vector b
    and the function which will be a normal backward sustitusion by default,
    the function of this parameter will be dealing with the second part of fthe solving
    

    reminder: we are searching for x, not y
    '''

    assert len(b) >= 1, 'this method isnt valid for solving multiply ecuation systems'

    #forward sustitutuon
    # Ux = y -> Solve Ly = b

    L_copy = np.copy(L)
    b_copy = np.copy(b)
    y = np.zeros(b.shape)

    for i in range(len(L)):
        total_sum = 0
        for j in range(i, -1, -1):
            total_sum += L_copy[i, j] * y[j]
            # print(f'total sum = {total_sum}')

        y[i] = (b_copy[i] - total_sum) / L_copy[i, i]
        # print(f'y = {y}')

    #backward sustitution
    # Ux = y
    solution = func(U, y)

    return solution



A = np.array([[4, -2, 1],
             [-2, 4, -2],
             [1, -2, 4]], dtype = float)

b = np.array([11, -16, 17])

B = np.array([[-14, 22], 
              [36, -18],
              [6, 7]])


L1, U1 = LU_decomposition(A)

sol = forward_backward_sustitution(L1, U1, b)
sol


array([ 1., -2.,  3.])

### Descomposición Choleski

In [31]:
def choleski_decomposition(A):
    '''
    the input is a symetric and positive defined matrix A, otherwise an assert will raise an error.

    teh output i matrix L

    A = L*L.T
    '''

    if not np.array_equal(A, A.T):
        raise ValueError("matrix A isnt symetric")
    
    if not np.all(np.linalg.eigvals(A) > 0):
        raise ValueError("matriz A isnt positive defined")

    L = np.zeros(A.shape)

    for i in range(0, len(A)):
        for j in range(i+1):
            total_sum = 0
            for k in range(j):
                total_sum += L[i, k] * L[j, k]
                
            if i == j: 
                L[j, j] = (A[i, j] - total_sum) ** 0.5
            else: 
                L[i, j] = (A[i, j] - total_sum) / L[j, j]
    return L



In [32]:
A = np.array([[1.44, -0.36, 5.52, 0.00],
            [-0.36, 10.33, -7.78, 0.00],
            [5.52, -7.78, 28.40, 9.00],
            [0.00, 0.00, 9.00, 61.00]], dtype=float)

b = np.array([0.04, -2.15, 0.00, 0.88], dtype=float)

L_13 = choleski_decomposition(A)
print(L_13)

forward_backward_sustitution(L_13, L_13.T, b)

[[ 1.2  0.   0.   0. ]
 [-0.3  3.2  0.   0. ]
 [ 4.6 -2.   1.8  0. ]
 [ 0.   0.   5.   6. ]]


array([ 3.09212567, -0.73871706, -0.8475723 ,  0.13947788])

### Descomposición y Resolución LU matriz tridiagonal

#### descomposición

In [33]:
def LU_tridiagonal_matrix(A):

    ''' 
    the input is a tridiagonal matrix A

    the output is a copy of the matrix A containing the coefficients updated by a lambda on the main diagonal
    and the corresponding lambdas on the lower diagonal.
    its a compact way of representing the LU to save memory storage.
    '''

    A_copy = np.copy(A)
    d = A.diagonal(0).copy()  # diag princ
    c = A.diagonal(-1).copy()  #diag inf
    e = A.diagonal(1).copy() #upper diag

    # print(d)
    # print(c)
    # print(e)

    n = len(d) 

    for k in range(1, n):
        lamb = c[k - 1] / d[k - 1]
        c[k - 1] = lamb
        d[k] = d[k] - lamb * e[k - 1]

    A_copy[np.arange(n), np.arange(n)] = d 
    A_copy[np.arange(1, n), np.arange(n-1)] = c    
    A_copy[np.arange(n-1), np.arange(1, n)] = e

    return A_copy

In [34]:
A = np.array([[4, 1, 0, 0],
            [2, 4, 1, 0],
            [0, 3, 4, 2],
            [0, 0, 1, 3]], dtype=float)

LU_tridiagonal_matrix(A)

array([[4.        , 1.        , 0.        , 0.        ],
       [0.5       , 3.5       , 1.        , 0.        ],
       [0.        , 0.85714286, 3.14285714, 2.        ],
       [0.        , 0.        , 0.31818182, 2.36363636]])

#### resolución

In [35]:
def solve_LU_tridiagonal_matrix(LU, b):
    '''
    the input is a matriz decomposition LU, and the solution vector b

    returns vector x with the solutions
    '''
    d = LU.diagonal(0).copy()  # diag princ
    c = LU.diagonal(-1).copy()  #diag inf
    e = LU.diagonal(1).copy() #upper diag

    y = np.zeros(d.shape)
    n = len(d)

    y[0] = b[0]
    for k in range(1, n):
        y[k] = b[k] - c[k-1] * y[k-1]

    x = np.zeros(d.shape)
    x[-1] = y[-1] / d[-1]
    for k in range(n-2, -1, -1):
        x[k] = (y[k] - e[k] * x[k+1]) / d[k]

    return x

In [36]:
A = np.array([[4, 1, 0, 0],
            [2, 4, 1, 0],
            [0, 3, 4, 2],
            [0, 0, 1, 3]], dtype=float)

b = np.array([15, 8, 22, 10], dtype=float)

LU = LU_tridiagonal_matrix(A)

print(f' el resultado por numpy es = \n {np.linalg.solve(A, b)}')

solve_LU_tridiagonal_matrix(LU, b)

 el resultado por numpy es = 
 [ 4.14423077 -1.57692308  6.01923077  1.32692308]


array([ 4.14423077, -1.57692308,  6.01923077,  1.32692308])

In [37]:
# forma programada de gpt

def LDL_decomposition(A):
    """
    Realiza la descomposición LDL^T de una matriz simétrica A.
    Retorna las matrices L y D.
    """
    n = A.shape[0]
    L = np.eye(n)  # Inicializamos L como la identidad
    D = np.zeros((n, n))  # Inicializamos D como una matriz diagonal

    for j in range(n):
        # Calcular D[j, j]
        D[j, j] = A[j, j] - sum(L[j, k]**2 * D[k, k] for k in range(j))
        
        for i in range(j + 1, n):
            # Calcular L[i, j]
            L[i, j] = (A[i, j] - sum(L[i, k] * L[j, k] * D[k, k] for k in range(j))) / D[j, j]
    
    return L, D


# Ejemplo de uso
A = np.array([
    [4, -2, 1, 0],
    [-2, 4, -2, 1],
    [1, -2, 4, -2],
    [0, 1, -2, 4]
], dtype=float)

L, D = LDL_decomposition(A)
L, D


(array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [-0.5       ,  1.        ,  0.        ,  0.        ],
        [ 0.25      , -0.5       ,  1.        ,  0.        ],
        [ 0.        ,  0.33333333, -0.5       ,  1.        ]]),
 array([[4.        , 0.        , 0.        , 0.        ],
        [0.        , 3.        , 0.        , 0.        ],
        [0.        , 0.        , 3.        , 0.        ],
        [0.        , 0.        , 0.        , 2.91666667]]))

### Pivotamiento Gauss 

In [30]:
#como dato no me acuerdo que cojones hice ni porque tengo qu evolver a definir estas funciones un poco cambiadas para que funciones perobueno funciona y punto 

def swap_rows(A, i , j):
    '''intercambia la fija j por la i 
    en la matriz de coeficientes A
    de un sistema lienal
    return una nueva matriz A con las filas intercambiadad
    pero con la matriz A original sin modificar'''

    A[[i, j]] = A[[j, i]]

    return A

def row_lineal_comb(A, i, j, k):

    A[j] = A[j] - A[i] * k

    return A

In [31]:
def gauss_pivot(A, b, tol=1.0e-12, func1 = swap_rows, func2= row_lineal_comb, func3= backward_sustitusion):

    n = len(A)
    
    s = np.amax(np.abs(A), axis= 1)

    # print('s = ', s)

    for k in range(n-1):
        #print(f'k = {k}')

        r = [np.abs(A[i, k]) / s[i] for i in range(k, len(s))]
        #print(f'r = {r}')
        p = np.argmax(r) + k
        #print(f'p = {p}')


        if p != k:
            A = func1(A, k, p)
            #print(f'A swap rows {k, p} = \n{A}')
            b = func1(b, k, p)
            #print(f'b swap rows {k, p} = {b}')
            s = func1(s, k, p)
            #print(f's swap rows {k, p} = {s}')

            for i in range(k+1, n):
                #print(f'i = {i}')
                c = A[i, k] / A[k, k]
                #print(f'c = {c}')
                A = func2(A, k , i, c)
                #print(f'A = {A}')
                b[i] -= c * b[k]
                #print(f'b = {b}')
    print(f'A =\n{A}')
    print(f'b = {b}')
    sol = func3(A, b)

    return sol

In [32]:
#ejemplo uso

A = np.array([
    [2, -2, 6],
    [-2, 4, 3],
    [-1, 8, 4]
], dtype=float)

b = np.array([16, 0, -1], dtype=float)

gauss_pivot(A, b)


A =
[[-2.          4.          3.        ]
 [ 0.          6.          2.5       ]
 [ 0.          0.          8.16666667]]
b = [ 0.         -1.         16.33333333]


array([ 1., -1.,  2.])

### Pivotamiento LU

In [34]:
def gauss_LU_pivot(A, b, tol=1.0e-12, func1 = swap_rows, func2= row_lineal_comb, func3= forward_backward_sustitution):

    n = len(A)
    
    s = np.amax(np.abs(A), axis= 1)

    seq = np.arange(n)

    L = np.eye((n))

    for k in range(n-1):
        r = [np.abs(A[i, k]) / s[i] for i in range(k, len(s))]
        p = np.argmax(r) + k

        if p != k:
            A = func1(A, k, p)
            seq = func1(seq, k, p)
            s = func1(s, k, p)

            for i in range(k+1, n):
                c = A[i, k] / A[k, k]
                L[i, k] = c
                A = func2(A, k , i, c)
                
    print(L)
    print(A)
    b_reordered = b[seq]
    print(b_reordered)
    U = A   
    sol = func3(L, U, b_reordered)

    return sol


A = np.array([
    [2, -2, 6],
    [-2, 4, 3],
    [-1, 8, 4]
], dtype=float)

b = np.array([16, 0, -1], dtype=float)

gauss_LU_pivot(A, b)

[[ 1.          0.          0.        ]
 [-1.          1.          0.        ]
 [ 0.5         0.33333333  1.        ]]
[[-2.          4.          3.        ]
 [ 0.          6.          2.5       ]
 [ 0.          0.          8.16666667]]
[ 0. -1. 16.]


array([ 1., -1.,  2.])

## resolucion hacia delante y hacia atras y combinarlo

esta arriba la implementacion hecha por mi


como dato mi codigo solo funcionaba para resolucion de un unico sistema. este codigo correge esos problemans y lo hace de forma genral 

In [16]:
import numpy as np

In [17]:
#gpt version, pendiente de checkearlo

def solve_backward_substitution(U, c):
    '''
    U: upper triangular matrix
    c: column vector or matrix (result from forward substitution)
    '''
    # Si c es un vector, convertirlo en 2D para unificar
    if len(c.shape) == 1:
        c = c.reshape(-1, 1)

    n = c.shape[0]
    X = np.zeros_like(c)  # Matriz para soluciones

    for k in range(c.shape[1]):  # Resolver columna por columna
        b = c[:, k]  # Tomar cada columna como un vector
        x = np.zeros(n)
        for i in range(n - 1, -1, -1):
            total_sum = sum(U[i, j] * x[j] for j in range(i + 1, n))
            x[i] = (b[i] - total_sum) / U[i, i]
        X[:, k] = x

    return X


def solve_forward_substitution(L, B):
    '''
    L: Matriz triangular inferior
    B: Matriz de soluciones o vector (si es un vector, se convierte en matriz de una sola columna)
    Retorna la matriz Y que satisface Ly = B
    '''
    # Convertir B en matriz si es un vector
    if len(B.shape) == 1:
        B = B.reshape(-1, 1)

    # Inicializamos Y
    Y = np.zeros_like(B)

    # Resolución hacia adelante para cada columna
    for k in range(B.shape[1]):
        b = B[:, k]  # Tomar la columna k como vector
        y = np.zeros_like(b)
        for i in range(len(L)):
            total_sum = sum(L[i, j] * y[j] for j in range(i))
            y[i] = (b[i] - total_sum) / L[i, i]
        Y[:, k] = y  # Guardar la solución de la columna

    return Y

def solve_forward_backward_substitution(L, U, B, func1 = solve_forward_substitution,func2=solve_backward_substitution):
    '''
    L: Matriz triangular inferior
    U: Matriz triangular superior
    B: Matriz de soluciones o vector
    func: Función para la sustitución hacia atrás (por defecto backward_sustitution)
    Retorna la matriz X que satisface LUx = B
    '''
    # Resolver hacia adelante: Ly = B
    Y = func1(L, B)

    # Resolver hacia atrás: Ux = Y
    X = func2(U, Y)

    return X



In [19]:
# Matriz A
A = np.array([[4, 2, 0],
              [2, 4, 2],
              [0, 2, 4]])

# Vector b o matriz B (varias ecuaciones simultáneas)
B = np.array([[2, 3],
              [4, 6],
              [6, 9]])

# Descomposición LU
L, U = LU_decomposition(A)

# Resolver sistema completo
X = solve_forward_backward_substitution(L, U, B)
print("Solución:\n", X)

Solución:
 [[0 1]
 [0 0]
 [2 3]]


### metodo de thomas para matrices tridiagonales

In [ ]:
import numpy as np

def thomas_algorithm(a, b, c, d):
    """
    Método de Thomas para resolver sistemas de ecuaciones con matrices tridiagonales.

    Parámetros:
    a: array (n-1) - elementos debajo de la diagonal principal (a_1 no existe)
    b: array (n)   - elementos de la diagonal principal
    c: array (n-1) - elementos encima de la diagonal principal (c_n no existe)
    d: array (n)   - términos independientes

    Retorna:
    x: array (n) - solución del sistema
    """
    n = len(b)
    # Copias de b y d para no modificar los originales
    c_ = np.zeros(n-1)
    d_ = np.zeros(n)

    # Fase de eliminación hacia adelante
    c_[0] = c[0] / b[0]
    d_[0] = d[0] / b[0]

    for i in range(1, n-1):
        temp = b[i] - a[i-1] * c_[i-1]
        c_[i] = c[i] / temp
        d_[i] = (d[i] - a[i-1] * d_[i-1]) / temp

    d_[n-1] = (d[n-1] - a[n-2] * d_[n-2]) / (b[n-1] - a[n-2] * c_[n-2])

    # Fase de sustitución hacia atrás
    x = np.zeros(n)
    x[n-1] = d_[n-1]

    for i in range(n-2, -1, -1):
        x[i] = d_[i] - c_[i] * x[i+1]

    return x

# Ejemplo de uso
a = [1, 1]               # Elementos debajo de la diagonal principal
b = [4, 4, 4]            # Diagonal principal
c = [1, 1]               # Elementos encima de la diagonal principal
d = [6, 6, 6]            # Términos independientes

sol = thomas_algorithm(a, b, c, d)
print("La solución del sistema es:", sol)
